# MNIST
## Importing the necessary package and datasets

From Yann LeCun, using the mnist data to build an accurate model that can recognize the handwriting data.

In [62]:
import numpy as np 
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#input data already been processed and able to be extract
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

# Outline the model 

In [66]:
input_size = 784
output_size = 10
hidden_layer_size = 100
#can put any hidden layer, but 50 is most used

tf.reset_default_graph() #it clears memory of all variables, reset all computational graph

#declare place holders
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

#tf.get_variable used to declare variable and using Xavier initialization
weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

#Creating HL, using tf.nn as it as the first neural network support. 
#Commonly used activation functions

outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])

outputs = tf.matmul(outputs_2, weights_3) + biases_3

#using softmax cross with logits as it applies softmax activation and calculate entropy loss
#using small number as small number will jeopardize our functions for outputs that has small probability
#Declaring LOSS and OPTIMIZING Algos
loss = tf.nn.softmax_cross_entropy_with_logits (logits = outputs, labels=targets)

mean_loss = tf.reduce_mean(loss)

#uses adam optimizer instead of gradient descent
optimize  = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(mean_loss)

#calculate accuracy using argmax
out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

#batch size = SGD(gradient descent), close to zero but reasonably high
batch_size = 100 #a hyperparameter
batches_number = mnist.train._num_examples // batch_size
max_epochs = 15 #we will change this once we found a sweeet spot

prev_validation_loss = 99999999.

#learning part of ml and a big loop of epoch
for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    #preloop across epoch in batches
    for batch_counter in range(batches_number):
        input_batch, target_batch = mnist.train.next_batch(batch_size) #built in next_batch
        _, batch_loss = sess.run([optimize, mean_loss], feed_dict = {inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss
        #calculate sum of each batches ^
        
    curr_epoch_loss /= batches_number #this is the training loss
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    #forward propagate to check the validatio below
    validation_loss , validation_accuracy = sess.run([mean_loss, accuracy], 
                                                    feed_dict = {inputs: input_batch, targets: target_batch})
    
    print('Epoch ' + str(epoch_counter + 1) + '. Training Loss: '+' {0:.3f}'.format(curr_epoch_loss)+
         '.Validation loss: '+'{0:.3f}'.format(validation_loss)+
         '.Validation Accuracy: '+'{0:.2f}'.format(validation_accuracy * 100)+ '%')
    
    if validation_loss > prev_validation_loss:
        break
    prev_validation_loss = validation_loss
    
print('End of training')
    
#validation is done by forward propagating it

C:\Users\George\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1. Training Loss:  0.324.Validation loss: 0.153.Validation Accuracy: 95.36%
Epoch 2. Training Loss:  0.132.Validation loss: 0.108.Validation Accuracy: 96.80%
Epoch 3. Training Loss:  0.094.Validation loss: 0.090.Validation Accuracy: 97.52%
Epoch 4. Training Loss:  0.072.Validation loss: 0.086.Validation Accuracy: 97.22%
Epoch 5. Training Loss:  0.056.Validation loss: 0.088.Validation Accuracy: 97.28%
End of training


Exercise:

Try changing the hyperparameters with width, depth, learning rate, and the relu stuff. 

Need test to prove its accurate as validation is not a good way to test accuracy


# Test the model

In [67]:
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)

test_accuracy = sess.run([accuracy], feed_dict = {inputs: input_batch, targets: target_batch})
test_accuracy_percent = test_accuracy[0] * 100
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent) + '%')

Test accuracy: 97.38%
